<a href="https://colab.research.google.com/github/mnaylor5/combined-bert-bow/blob/main/notebooks/Apply_Normalization_to_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prep Data for Modeling
This started out as a copy of the modeling notebook, but I'm repurposing it for the normalization code. 

Install Git LFS and clone our repo

In [1]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs && git lfs install
!git clone https://github.com/mnaylor5/combined-bert-bow.git

Detected operating system as Ubuntu/bionic.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 104 not upgraded.
Need to get 6,229 kB of archives.
After this operation, 14.5 MB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu bionic/main amd64 git-lfs amd64 2.13.3 [6,229 kB]
Fetched 6,229 kB in 1s (10.1 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/pe

In [2]:
import pandas as pd
import string
import nltk
nltk.download('popular')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

# Pre-processing of raw data
Starting with the dataframe Vedant put in Git (first PR)


In [3]:
text = pd.read_csv(r'combined-bert-bow/data/CaptumAttributionDF.csv')
MAX_LEN = 512
texts = pd.DataFrame(text)
texts.head()

,label,text,text_norm
0,1,My dad is hospitalized and had to be put in a ...,"['', '', '', '', '', '', '', '', '', '', 'h', ..."
1,1,Feels like anxiety is turning my brain to stew...,"['F', 'e', 'e', 'l', '', '', 'l', '', 'k', 'e'..."
2,1,What do you do when anxiety kicks in around ot...,"['W', 'h', '', '', '', '', '', '', '', '', 'u'..."
3,1,Finding a job is a nightmare I want to rant ab...,"['F', '', 'n', '', '', 'n', 'g', '', '', '', '..."
4,1,All I want is to be able to sleep again... I g...,"['', 'l', 'l', '', '', '', 'w', '', 'n', '', '..."


## Create a Bag of Words
The three steps to do so are:


1.   Remove punctuation
2.   Lemmatize the text
3.   Remove stopwords


In [4]:
import re
lemmatizer = WordNetLemmatizer()
STOPS = stopwords.words('english') # already lowercased and split contractions
def normalize_text(x, stop_words=STOPS, min_word_len=2):
  # lowercase
  x = x.lower()

  # remove punctuation (everything but letters or spaces)
  x = re.sub(r'[^a-z\s]', ' ', x)

  # tokenize by spacing and remove stop words / super short words
  tokens = re.split(r'\s+', x)
  tokens = filter(lambda x: (x not in stop_words) & (len(x) > min_word_len),
                  tokens) 
  
  # lemmatize now
  tokens = map(lemmatizer.lemmatize, tokens)
  
  return ' '.join(tokens)

In [5]:
text['text'].head(10).apply(normalize_text).values

array(['dad hospitalized put breathing machine help breathe trying keep together family life asia usa found last night dad hospitalized due hard time breathe escalated breathing tube mom freaking trying positive anxiety high low fly asia mid week giving anxiety added patiently waiting work grant leave know monday part want fly think could handle seeing hooked ventilator mom putting weight shoulder taking care everything saturday agony waiting till monday take care necessary paperwork get approved causing much anxiety trying positive everything thought hooked ventilator causing shut hoping progress recovery get',
       'feel like anxiety turning brain stew lately fucking stew emotion anxiety consider emotional person jesus christ brain feel like jumble emotion anxious rumination fog make sense every night basically turning worry session sun rise lay thinking shitty feel everything life could wrong shit thinking people care probably hate reason hardly explain remember acting like dumbas

In [6]:
%%time
text['text_norm'] = text['text'].apply(normalize_text)

CPU times: user 2min 32s, sys: 659 ms, total: 2min 33s
Wall time: 2min 33s


Save the data

In [7]:
text.to_csv('normalized_depression_text.csv', index=False)